In [13]:
import pandas as pd
import xarray as xr
import os
import subprocess
import seaborn as sns
import matplotlib.pyplot as plt

import allel

import rasterio
import pandas as pd
import zipfile
import os
import numpy as np


worldclim has monthly data for different periods
https://www.worldclim.org/data/monthlywth.html
the granularity is not great, but it is the easiest way to get climate data


In [5]:
accessions = pd.read_csv('../meta_filtered_clusters.csv')

In [6]:
accessions

,accession,year,lat,lon,country,type,cluster
0,PL0012,1881.0,55.69076,13.165520,SWE,historical,Scandinavia
1,PL0109,1881.0,56.18390,15.609250,SWE,historical,Scandinavia
2,PL0119,1884.0,55.54033,13.271690,SWE,historical,Scandinavia
3,PL0103,1889.0,55.56733,12.966290,SWE,historical,Scandinavia
4,LND0702ad,1891.0,57.93100,12.532180,SWE,historical,Scandinavia
...,...,...,...,...,...,...,...
373,9997,1999.0,48.56000,9.160000,GER,modern,CentralEurope
374,9999,1999.0,48.53000,9.060000,GER,modern,CentralEurope
375,10001,1999.0,48.52000,9.050000,GER,modern,CentralEurope
376,10022,1999.0,48.03330,7.766700,GER,modern,CentralEurope


In [2]:
os.listdir('wc/')

['wc2.1_cruts4.09_2.5m_tmin_1970-1979.zip',
 'wc2.1_cruts4.09_2.5m_prec_1970-1979.zip',
 'wc2.1_cruts4.09_2.5m_tmax_1990-1999.zip',
 'wc2.1_cruts4.09_2.5m_tmin_1990-1999.zip',
 'wc2.1_cruts4.09_2.5m_prec_1990-1999.zip',
 'wc2.1_cruts4.09_2.5m_tmax_1970-1979.zip']

In [28]:
import tempfile
import shutil

def read_raster_from_zip(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as z:
        tif_file = [f for f in z.namelist() if f.endswith('.tif')][0]
        with tempfile.TemporaryDirectory() as tmpdir:
            extracted_path = z.extract(tif_file, path=tmpdir)
            with rasterio.open(extracted_path) as src:
                data = src.read(1)
                transform = src.transform
                crs = src.crs
            return data, transform, crs


In [29]:
def extract_value(data, transform, lon, lat):
    row, col = ~transform * (lon, lat)
    row, col = int(row), int(col)
    if 0 <= row < data.shape[0] and 0 <= col < data.shape[1]:
        return data[row, col]
    else:
        return np.nan


In [30]:
# Load accessions
accessions = pd.read_csv('../meta_filtered_clusters.csv')

# Map for each variable and time window
file_map = {
    ('historical', 'tmin'): 'wc/wc2.1_cruts4.09_2.5m_tmin_1970-1979.zip',
    ('historical', 'tmax'): 'wc/wc2.1_cruts4.09_2.5m_tmax_1970-1979.zip',
    ('historical', 'prec'): 'wc/wc2.1_cruts4.09_2.5m_prec_1970-1979.zip',
    ('modern', 'tmin'): 'wc/wc2.1_cruts4.09_2.5m_tmin_1990-1999.zip',
    ('modern', 'tmax'): 'wc/wc2.1_cruts4.09_2.5m_tmax_1990-1999.zip',
    ('modern', 'prec'): 'wc/wc2.1_cruts4.09_2.5m_prec_1990-1999.zip'
}

In [19]:
results = []

for idx, row in accessions.iterrows():
    acc = row['accession']
    lat = row['lat']
    lon = row['lon']
    sample_type = row['type']

    # Extract each variable
    vals = {'accession': acc}
    for var in ['tmin', 'tmax', 'prec']:
        zip_path = file_map[(sample_type, var)]
        data, transform, crs = read_raster_from_zip(zip_path)
        val = extract_value(data, transform, lon, lat)
        vals[f'{var}_{sample_type}'] = val
    results.append(vals)

climate_df = pd.DataFrame(results)

In [20]:
climate_df

,accession,tmin_historical,tmax_historical,prec_historical,tmin_modern,tmax_modern,prec_modern
0,PL0012,NaN,NaN,NaN,NaN,NaN,NaN
1,PL0109,NaN,NaN,NaN,NaN,NaN,NaN
2,PL0119,NaN,NaN,NaN,NaN,NaN,NaN
3,PL0103,NaN,NaN,NaN,NaN,NaN,NaN
4,LND0702ad,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
373,9997,NaN,NaN,NaN,NaN,NaN,NaN
374,9999,NaN,NaN,NaN,NaN,NaN,NaN
375,10001,NaN,NaN,NaN,NaN,NaN,NaN
376,10022,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
results = []

# Helper function to build the correct filename based on type and variable
def get_zip_path(sample_type, var):
    if sample_type == 'historical':
        return f"wc/wc2.1_cruts4.09_2.5m_{var}_1970-1979.zip"
    elif sample_type == 'modern':
        return f"wc/wc2.1_cruts4.09_2.5m_{var}_1990-1999.zip"
    else:
        return None

# Main loop
for idx, row in accessions.head(2).iterrows():
    acc = row['accession']
    lat = row['lat']
    lon = row['lon']
    sample_type = row['type']
    
    vals = {'accession': acc}

    for var in ['tmin', 'tmax', 'prec']:
        zip_path = get_zip_path(sample_type, var)
        if zip_path is None:
            continue
        try:
            data, transform, crs = read_raster_from_zip(zip_path)
            val = extract_value(data, transform, lon, lat)
            vals[f"{var}_{sample_type}"] = val
        except Exception as e:
            print(f"Failed to extract {var} for {acc}: {e}")
            vals[f"{var}_{sample_type}"] = None
    
    results.append(vals)

climate_df = pd.DataFrame(results)


In [32]:
data, transform, crs = read_raster_from_zip(zip_path)

In [34]:
data

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)

In [33]:
extract_value(data, transform, lon, lat)

nan

In [22]:
climate_df

,accession,tmin_historical,tmax_historical,prec_historical
0,PL0012,NaN,NaN,NaN
1,PL0109,NaN,NaN,NaN


In [23]:
import zipfile
import rasterio

def read_raster_from_zip(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as z:
        # List all .tif files inside the zip
        tif_files = [f for f in z.namelist() if f.endswith('.tif')]
        if not tif_files:
            raise FileNotFoundError("No .tif file found in zip.")
        with z.open(tif_files[0]) as tif:
            with rasterio.open(tif) as src:
                return src.read(1), src.transform, src.crs

In [24]:
def extract_value(data, transform, lon, lat):
    import rasterio
    from rasterio.transform import rowcol

    try:
        row, col = rowcol(transform, lon, lat)
        if 0 <= row < data.shape[0] and 0 <= col < data.shape[1]:
            return data[row, col]
        else:
            return None
    except Exception as e:
        return None


In [25]:
print(f"{acc}: type={sample_type}, lat={lat}, lon={lon}, file={zip_path}")


PL0109: type=historical, lat=56.1839, lon=15.60925, file=wc/wc2.1_cruts4.09_2.5m_prec_1970-1979.zip


In [26]:
lat = 55.69076
lon = 13.16552
sample_type = 'historical'
zip_path = get_zip_path(sample_type, 'tmin')

data, transform, crs = read_raster_from_zip(zip_path)
val = extract_value(data, transform, lon, lat)
print(val)

SystemError: <class 'rasterio._err.CPLE_OpenFailedError'> returned a result with an error set

In [35]:
import os
import glob
import rasterio
import pandas as pd

def extract_monthly_values(tif_files, lon, lat):
    monthly_values = []
    for tif_file in sorted(tif_files):
        with rasterio.open(tif_file) as src:
            row, col = src.index(lon, lat)
            value = src.read(1)[row, col]
            monthly_values.append(value)
    return monthly_values

# Prepare results container
results = []

# Loop over ecotypes
for idx, row in accessions.head(2).iterrows():
    
    acc = row['accession']
    lat = row['lat']
    lon = row['lon']
    print(acc, lat, lon)
    record = {'accession': acc}

    for climate_type, decade in [('historical', '1970-1979'), ('modern', '1990-1999')]:
        for var in ['tmin', 'tmax', 'prec']:
            path_pattern = f"~/scratch/selection_scan/climate/wc/wc2.1_cruts4.09_2.5m_{var}_{decade}_*.tif"
            tif_files = glob.glob(os.path.expanduser(path_pattern))
            if not tif_files:
                continue
            values = extract_monthly_values(tif_files, lon, lat)
            for i, val in enumerate(values):
                record[f"{var}_{climate_type}_month{i+1:02d}"] = val

    results.append(record)

climate_monthly_df = pd.DataFrame(results)


In [49]:
import pandas as pd
import rasterio
import os
from glob import glob
from collections import defaultdict

# Setup
data_dir = "/carnegie/nobackup/scratch/tbellagio/selection_scan/climate/wc"
variables = ["tmin", "tmax", "prec"]
periods = {
    "historical": list(range(1970, 1980)),
    "modern": list(range(1990, 2000))
}
months = [f"{i:02d}" for i in range(1, 13)]

# Result container
results = []

# Iterate over accessions
for idx, row in accessions.head(2).iterrows():
    acc = row["accession"]
    lat = row["lat"]
    lon = row["lon"]
    acc_record = {"accession": acc}
    print(acc, lat, lon)
    for period_name, years in periods.items():
        for var in variables:
            # Matrix to collect values by month
            monthly_values = defaultdict(list)

            for year in years:
                for month in months:
                    tif_name = f"wc2.1_cruts4.09_2.5m_{var}_{year}-{month}.tif"
                    tif_path = os.path.join(data_dir, tif_name)
                    if not os.path.exists(tif_path):
                        continue

                    try:
                        with rasterio.open(tif_path) as src:
                            row_idx, col_idx = src.index(lon, lat)
                            val = src.read(1)[row_idx, col_idx]
                            if val is not None:
                                monthly_values[month].append(val)
                    except Exception as e:
                        print(f"Error reading {tif_path} for {acc}: {e}")
                        continue

            # Average across years for each month
            for month in months:
                vals = monthly_values[month]
                avg_val = sum(vals) / len(vals) if vals else None
                acc_record[f"{var}_{period_name}_month{month}"] = avg_val

    results.append(acc_record)

# Convert to DataFrame
climate_monthly_df = pd.DataFrame(results)


PL0012 55.69076 13.16552
PL0109 56.1839 15.60925


In [50]:
climate_monthly_df

,accession,tmin_historical_month01,tmin_historical_month02,tmin_historical_month03,tmin_historical_month04,tmin_historical_month05,tmin_historical_month06,tmin_historical_month07,tmin_historical_month08,tmin_historical_month09,...,prec_modern_month03,prec_modern_month04,prec_modern_month05,prec_modern_month06,prec_modern_month07,prec_modern_month08,prec_modern_month09,prec_modern_month10,prec_modern_month11,prec_modern_month12
0,PL0012,-2.1,-2.5,-0.5,1.9,6.7,10.9,12.8,12.5,9.4,...,36.4,37.84,44.589999,53.44,53.130000,55.570000,73.569999,59.94,57.279999,57.790001
1,PL0109,-3.0,-3.7,-1.9,0.7,5.3,9.8,12.0,11.6,8.2,...,30.3,34.51,40.380000,48.88,49.889999,53.370001,76.080000,55.89,64.200001,58.530000


In [43]:
tif_name

'wc2.1_cruts4.09_2.5m_prec_1999-12.tif'

In [46]:
pwd

'/carnegie/nobackup/scratch/tbellagio/selection_scan/climate'

In [47]:
tif_path = 'wc/wc2.1_cruts4.09_2.5m_prec_1999-12.tif'

In [48]:
with rasterio.open(tif_path) as src:
    row_idx, col_idx = src.index(lon, lat)
    val = src.read(1)[row_idx, col_idx]
    if val is not None:
        monthly_values[month].append(val)

In [38]:
decade

'1990-1999'

In [ ]:
climate_prec.to_csv('climate_prec_ecotypes.csv')
climate_max_temp.to_csv('climate_max_temp_ecotypes.csv')
climate_min_temp.to_csv('climate_min_temp_ecotypes.csv')

In [ ]:
## run the dismo package from R to get the bioclim variables
subprocess.call("~/miniforge3/envs/r-environment/bin/Rscript --vanilla  /home/tbellagio/scratch/climate_grenet/run_dismo_bioclim_calc_ecotypes.R", 
                shell=True)